<a href="https://colab.research.google.com/github/singhvis29/Ads_by_Ini_Projects/blob/main/ABI_Video1_Impressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deep-sort-realtime


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import cv2
import numpy as np
import os

In [4]:
import math

from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

In [5]:
import os
import cv2
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection
from collections import defaultdict
from tqdm import tqdm
# from sort import Sort

In [6]:
from deep_sort_realtime.deepsort_tracker import DeepSort

### Read Video and save it as frames

In [7]:
video_path = '/content/drive/MyDrive/Projects/Ads_by_Ini/clip_1.mp4'
output_folder = '/content/drive/MyDrive/Projects/Ads_by_Ini/frames'

In [8]:


# # Create the output folder if it doesn't exist
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # Create a VideoCapture object
# cap = cv2.VideoCapture(video_path)

# # Check if video opened successfully
# if (cap.isOpened() == False):
#     print("Error opening video file")

# # Print number of frame
# print('Number of Frames:', cap.get(cv2.CAP_PROP_FPS))

# # Read until video is completed
# while(cap.isOpened()):
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     if ret == True:
#         # Construct the output file path
#         output_path = os.path.join(output_folder, 'frame_' + str(int(cap.get(cv2.CAP_PROP_POS_FRAMES))) + '.png')

#         # Save the resulting frame
#         cv2.imwrite(output_path, frame)

#         # Press Q on keyboard to exit
#         if cv2.waitKey(25) & 0xFF == ord('q'):
#             break
#     # Break the loop
#     else:
#         break

# # When everything done, release the video capture object
# cap.release()

# # Closes all the frames
# # cv2.destroyAllWindows() # This will not work in Colab directly

# print("Video processing complete.")

### Load Models

In [9]:
# model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
# model.eval();

### Impressions

In [ ]:
import os
import cv2
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection
from collections import defaultdict
from tqdm import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort

# --------------------------
# Device setup
# --------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# --------------------------
# Load pretrained DETR on GPU
# --------------------------
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50").to(device)

# --------------------------
# Initialize DeepSORT tracker
# --------------------------
tracker = DeepSort(
    max_age=30,
    n_init=2,
    nms_max_overlap=1.0,
    max_cosine_distance=0.2,
    embedder="mobilenet",
    half=torch.cuda.is_available(),  # use FP16 if GPU
)

# --------------------------
# Detection threshold + frames path
# --------------------------
THRESHOLD = 0.7
frames_folder = output_folder
frame_files = sorted(os.listdir(frames_folder))
FPS = 60.0 # Make sure this matches your video's FPS

# Store tracks
object_tracks = defaultdict(list)

# --------------------------
# Process each frame
# --------------------------
for frame_id, file in tqdm(enumerate(frame_files), total=len(frame_files)):
    frame_path = os.path.join(frames_folder, file)
    frame = cv2.imread(frame_path)

    # Preprocess & move to GPU
    inputs = processor(images=frame, return_tensors="pt").to(device)

    with torch.no_grad():  # disable autograd for faster inference
        outputs = model(**inputs)

    # Post-process still happens on CPU
    target_sizes = torch.tensor([frame.shape[:2]], device=device)
    results = processor.post_process_object_detection(
        outputs, target_sizes=target_sizes.cpu(), threshold=THRESHOLD
    )[0]

    # Prepare detections for DeepSORT: [[x1, y1, x2, y2], confidence, class_name]
    detections = []
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        cls = model.config.id2label[label.item()]
        xmin, ymin, xmax, ymax = box.tolist()
        # Ensure the bounding box is a list within the detection list
        detections.append([[xmin, ymin, xmax, ymax], float(score), cls])


    # Update DeepSORT
    tracks = tracker.update_tracks(detections, frame=frame)

    timestamp = frame_id / FPS

    # Collect track info
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        x1, y1, x2, y2 = track.to_tlbr()
        # Get class from the detection that led to this track update
        # This is a simplification; a more robust approach would store class with the track state
        det_class = "unknown"
        for det in detections:
            det_bbox = det[0]
            # Simple check if the detection bbox is close to the track bbox
            if abs(det_bbox[0] - x1) < 10 and abs(det_bbox[1] - y1) < 10: # Use a small tolerance
                det_class = det[2]
                break


        area = (x2 - x1) * (y2 - y1)
        distance_est = 1 / (area + 1e-6) # Simple inverse area as distance proxy

        object_tracks[(det_class, int(track_id))].append({
            "frame": frame_id,
            "time": timestamp,
            "bbox": [float(x1), float(y1), float(x2), float(y2)],
            "distance_est": distance_est
        })

# --------------------------
# Summarize grouped by class
# --------------------------
summary = defaultdict(list)

for (cls, track_id), entries in object_tracks.items():
    duration = (entries[-1]["time"] - entries[0]["time"]) if len(entries) > 1 else 1/FPS
    avg_distance = sum(e["distance_est"] for e in entries) / len(entries)
    summary[cls].append({"track_id": track_id, "duration_sec": duration, "avg_distance_est": avg_distance})

print("\nSummary grouped by class:")
for cls, objs in summary.items():
    num_objs = len(objs)
    avg_duration = sum(o["duration_sec"] for o in objs) / num_objs if num_objs > 0 else 0
    avg_distance = sum(o["avg_distance_est"] for o in objs) / num_objs if num_objs > 0 else 0
    print(f"{cls}: {num_objs} objects, avg duration {avg_duration:.2f}s, avg distance proxy {avg_distance:.4f}")

Using device: cuda


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas